In [22]:
import os
current_dir = '/Users/lpzatr/OneDrive/OneDrive UoN/OneDrive - The University of Nottingham/synched/python/dwi_tracing'
os.chdir(current_dir)

In [23]:
# Load configuration parameters from JSON file
import json

def _json_object_hook(d): return namedtuple('X', d.keys())(*d.values())
def json2obj(data): return json.loads(data, object_hook=_json_object_hook)

with open('config_cing.json', 'r') as myfile:
    json_string=myfile.read() #.replace('\n', '')

config = json.loads(json_string)
config_gen = config['general']
config_bpx = config['bedpostx']
config_ptx = config['probtrackx']

print(config)

if not os.path.isdir(config_gen['root_dir']):
    raise Exception('Root dir does not exist: {}'.format(config_gen['root_dir']))
    
raw_dir = '{0}/{1}'.format(config_gen['root_dir'], config_gen['raw_dir'])
deriv_dir = '{0}/{1}'.format(config_gen['root_dir'], config_gen['deriv_dir'])

{'general': {'fsl_bin': '/usr/local/fsl/bin/', 'root_dir': '/Volumes/AndrewElements/data/nki', 'raw_dir': 'raw', 'deriv_dir': 'derivatives', 'prefix': 'sub-', 'session': 'ses-DS2', 'subjects_file': 'subject_cing.list', 'clobber': False, 'dryrun': False}, 'bedpostx': {'anat_suffix': 'T1w', 'bval_suffix': 'bval', 'bvec_suffix': 'bvec', 'eddy_suffix': 'eddy', 'eddy_reference': 0, 'bet_suffix': 'bet', 'mni_suffix': 'space-MNI152lin', 'mni_ref': 'MNI152_${T1}_2mm', 'mean3g_ref': 'Mean3G.nii.gz', 'mean3g_mask': 'Mask.nii.gz', 'mean3g_config': 'G3_FAnormalization.cnf'}, 'qsub': {'use_qsub': False, 'walltime': '70:00:00', 'select': 1, 'ncpus': 10, 'mem': '40gb', 'project_id': 'HPCA-03714-IFR', 'temp_dir': '/Users/lpzatr/Documents/python/dwi_tracing/tmp', 'output_dir': '/Users/lpzatr/Documents/python/dwi_tracing/io'}, 'probtrackx': {'roi_dir': '/Volumes/AndrewElements/data/rois/cing_rs', 'roi_list': 'cing.rois', 'network_name': 'cing_rs', 'distthresh': 5.0, 'sampvox': 2, 'nsteps': 2000, 'cthr':

In [6]:
# Read subject names into subjects file
import csv
import os

os.chdir(current_dir)

print()

# Find only subjects with all folders
subjects0 = []
subdirs = next(os.walk(raw_dir))[1]

for x in subdirs:
    subsubs = next(os.walk('{0}/{1}/{2}'.format(raw_dir,x,config_gen['session'])))[1]
#     print(x);
    if len(subsubs) == 3:
        subjects0.append(x[4:])

print('Found {} subjects with data'.format(len(subjects0)))
        
subjects1 = []

with open('nki_diagnoses.csv', 'r') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        if row[0] in subjects0:
            subjects1.append(row[0])
      
subjects2 = []

with open('nki_ages.csv', 'r') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        subjects2.append(row[0])
        
# Intersect with age list
subjects=list(filter(lambda x: x in subjects1, subjects2))

with open('subjects_all.list','w') as subj_file:
    for subject in subjects:
        subj_file.write('{}\n'.format(subject))

print('Wrote {} subjects'.format(len(subjects)))


Found 300 subjects with data
Wrote 135 subjects


In [7]:
# Preliminary stuff

# Create derivatives directory if it doesn't exist
#os.chdir(config['root_dir'])

if not os.path.isdir(deriv_dir):
    os.mkdir(deriv_dir)
    print('Created derivatives dir: {}'.format(deriv_dir))

In [8]:
import subprocess
from subprocess import Popen 

def run_fsl(cmd):
    if config_gen['dryrun']:
        return ''
    sp = Popen(cmd, shell=True, stderr=subprocess.PIPE)
    out, err = sp.communicate()
    return err

In [9]:
def process_subject(subject, config):
    
    # Step 1: Eddy current correction
    # eddy_correct <4dinput> <4doutput> <reference_no>
    config_gen = config['general']
    config_bpx = config['bedpostx']
    
    fsl_bin = config_gen['fsl_bin']
    
    subj = '{}{}'.format(config_gen['prefix'], subject)
    input_dir = '{0}/{1}/{2}/dwi' \
                        .format(raw_dir, subj, config_gen['session'])
    output_dir = '{0}/{1}/{2}/dwi' \
                        .format(deriv_dir, subj, config_gen['session'])
    if not os.path.isdir(output_dir):
        os.makedirs(output_dir)
    
    bedpostx_dir = '{0}/bedpostX' \
                    .format(output_dir)
    bedpostx_done = os.path.isdir(bedpostx_dir)
    
    if bedpostx_done and not config_gen['clobber']:
        print('\tBedpostX results already exist. Skipping those steps.')
    else:
    
        dwi_img = '{0}/data.nii.gz' \
                        .format(output_dir)

        input_img = '{0}/{1}_{2}_dwi.nii.gz' \
                        .format(input_dir, subj, config_gen['session'])

        output_img = '{0}/{1}_{2}_dwi_{3}.nii.gz' \
                        .format(output_dir, subj, config_gen['session'], config_bpx['eddy_suffix'])

        if os.path.exists(dwi_img) and not config_gen['clobber']:
            print('\tEddy output exists for {}. Skipping.'.format(subject))
        else:
            cmd_eddy = '{0}eddy_correct {1} {2} {3}' \
                        .format(fsl_bin, input_img, output_img, config_bpx['eddy_reference'])
#             print('\t{}'.format(cmd_eddy))
            err = run_fsl(cmd_eddy)
            if err:
                print('\tError with eddy correction [{0}]: {1}'.format(subject,err))
                return False
            
            print('\tDone eddy correction [{}].'.format(subject))   

            # Rename image to data.nii.gz
            cmd = 'mv {0} {1}' \
                            .format(output_img, dwi_img);
#             print('\t{}'.format(cmd))
            err = run_fsl(cmd)
            if err:
                print('\tError renaming eddy output image [{0}]: {1}'.format(subject,err))
                return False


        # Step 2: Brain extraction (BET)
        #      2.1: Extract B0 images (using fslroi and bvals file)
        #      2.2: Get mean B0 image (using fslmerge)
        #      2.3: Run BET on the mean B0

        # Check if output already exists
        bet_img = '{0}/{1}_{2}_dwi_{3}' \
                        .format(output_dir, subj, config_gen['session'], config_bpx['bet_suffix'])

        nodif_img = '{0}/nodif.nii.gz'.format(output_dir)

        if (os.path.exists(bet_img) or os.path.exists(nodif_img)) and not config_gen['clobber']:
            print('\tBET output exists for {}. Skipping.'.format(subject))
        else:

            input_img = dwi_img;
            bval_file = '{0}/{1}_{2}_dwi.bval' \
                            .format(input_dir, subj, config_gen['session'])

            bzeros = []
            with open(bval_file, 'r') as csvfile:
                reader = csv.reader(csvfile, delimiter=' ')
                for row in reader:
                    if len(row) > 0:
                        c = 0
                        for col in row:
                            if len(col) > 0 and int(col) < 100:
                                bzeros.append(c)
                            c+=1;

            # Extract each B0
            for i in range(len(bzeros)):
                cmd = '{0}fslroi {1} {2}/xno_{3}.nii.gz {4} 1' \
                            .format(fsl_bin, input_img, output_dir, i, bzeros[i])
#                 print('\t{}'.format(cmd))
                err = run_fsl(cmd)
                if err:
                    print('\tError extracting B0 images [{0}]: {1}'.format(subject,err))
                    return False

            # Merge into a 4D image
            input_img = nodif_img
            cmd = '{0}fslmerge -t {1} {2}/xno*.nii.gz' \
                            .format(fsl_bin, input_img, output_dir)
#             print('\t{}'.format(cmd))
            err = run_fsl(cmd)
            if err:
                print('\tError merging B0 images [{0}]: {1}'.format(subject,err))
                return False

            # Compute mean B0 from this image
            cmd = '{0}fslmaths {1} -Tmean {1}' \
                            .format(fsl_bin, input_img)
#             print('\t{}'.format(cmd))
            err = run_fsl(cmd)
            if err:
                print('\tError computing B0 image [{0}]: {1}'.format(subject,err))
                return False

            # Run BET on the output
            cmd_bet = '{0}bet2 {1} {2} -m -f 0.3' \
                        .format(fsl_bin, input_img, bet_img)
#             print('\t{}'.format(cmd_bet))
            err = run_fsl(cmd_bet)
            if err:
                print('\tError with brain extraction (BET) [{0}]: {1}'.format(subject,err))
                return False
            
            # Rename the BET mask
            cmd = 'mv {0}_mask.nii.gz {1}/nodif_brain_mask.nii.gz' \
                        .format(bet_img, output_dir)
#             print('\t{}'.format(cmd))
            err = run_fsl(cmd)
            if err:
                print('\tError removing residual B0 images [{0}]: {1}'.format(subject,err))
                return False

            bet_mask_img = '{0}/nodif_brain_mask.nii.gz'.format(output_dir)

            # Clean up the mess
            cmd = 'rm {0}/xno*.nii.gz; rm {0}/sub*'.format(output_dir);
            print('\t{}'.format(cmd))
            run_fsl(cmd) # Ignore any error here
            
            print('\tDone brain extraction (BET) [{}].'.format(subject))

        # Step 3: BedpostX (Fitting model)
        #      3.1: Copy bvals and bvecs
        #      3.2: Run dtifit
        #      3.3: Run BedpostX

        # Copy bvals and bvecs
        cmd = 'cp {0}/{1}_{2}_dwi.bvec {3}/bvecs; cp {0}/{1}_{2}_dwi.bval {3}/bvals' \
                        .format(input_dir, subj, config_gen['session'], output_dir)

        print('\t{}'.format(cmd))
        err = run_fsl(cmd)
        if err:
            print('\tError copying bvecs and bvals [{0}]: {1}'.format(subject,err))
            return False

        # Run dtifit
        if os.path.exists('{0}/dti_FA.nii.gz'.format(output_dir)) and not config_gen['clobber']:
            print('\tDTI output exists for {}. Skipping.'.format(subject))
        else:
            cmd = '{0}dtifit -k {1} -m {2} -r {3}/bvecs -b {3}/bvals -o {3}/dti' \
                        .format(fsl_bin, dwi_img, bet_mask_img, output_dir)
            print('\t{}'.format(cmd))
            err = run_fsl(cmd)
            if err:
                print('\tError running dtifit [{0}]: {1}'.format(subject,err))
                return False
           
            print('\tDone dtifit [{}].'.format(subject))

        # Run BedpostX
        cmd = '{0}bedpostx {1}' \
                    .format(fsl_bin, output_dir)
#         print('\t{}'.format(cmd))
        err = run_fsl(cmd)
        if err:
            print('\tError running BedpostX [{0}]: {1}'.format(subject,err))
            return False
        
        # Clean up
        # Move bedpost directory to proper location
        dummy_dir = '{0}/{1}/{2}/dwi.bedpostX' \
                        .format(deriv_dir, subj, config_gen['session'])
        cmd = 'mv {0} {1}' \
                        .format(dummy_dir, bedpostx_dir)
        print('\t{}'.format(cmd))
        err = run_fsl(cmd)
        if err:
            print('\tError running BedpostX [{0}]: {1}'.format(subject,err))
            return False

        output_img = '{0}/{1}_{2}_dwi_{3}.nii.gz' \
                        .format(output_dir, subj, config_gen['session'], config_bpx['bet_suffix'])

        cmd = 'rm {0}/nodif*; rm {0}/bv*; mv {0}/data.nii.gz {1}' \
                        .format(output_dir, output_img)
        print('\t{}'.format(cmd))
        err = run_fsl(cmd)
        if err:
            print('\tError cleaning up DWI folder [{0}]: {1}'.format(subject,err))
            return False
        
        print('\tDone BedpostX [{}].'.format(subject))
    
    # Step 4: Warping to template space
    #      4.1: Linear transform (FLIRT)
    #      4.2: Non-linear warp (FNIRT)
    #      4.3: Inverse NL warp (INVWARP)
    
    # Warp to Mean 3G
    reg3g_dir = '{0}/reg3G' \
                    .format(output_dir)
        
    if os.path.isdir(reg3g_dir) and not config_gen['clobber']:
        print('\tReg3D output exists for {}. Skipping.'.format(subject))
    else:
        os.mkdir(reg3g_dir)
        
        # Linear (FLIRT)
        cmd = '{0}flirt -in {1}/dti_FA.nii.gz ' \
                       '-ref utils/{2} ' \
                       '-out {3}/FA_lin2Mean3G.nii.gz ' \
                       '-omat {3}/FA_lin2Mean3G.mat ' \
                       '-bins 256 -cost corratio ' \
                       '-searchrx -180 180 -searchry -180 180 -searchrz -180 180 ' \
                       '-dof 12 -interp spline' \
                            .format(fsl_bin, output_dir, config_bpx['mean3g_ref'], reg3g_dir)
        
        print('\t{}'.format(cmd))
        err = run_fsl(cmd)
        if err:
            print('\tError warping to Mean3G (flirt) [{0}]: {1}'.format(subject,err))
            return False
        
        # Non-linear (FNIRT)
        cmd = '{0}fnirt --in={1}/dti_FA.nii.gz ' \
                       '--ref=utils/{2} ' \
                       '--refmask=utils/{3} ' \
                       '--aff={4}/FA_lin2Mean3G.mat ' \
                       '--config=utils/{5} ' \
                       '--cout={4}/FA_warp2Mean3G.nii.gz ' \
                       '--iout={4}/FA_nlin2Mean3G.nii.gz' \
                            .format(fsl_bin, output_dir, config_bpx['mean3g_ref'], \
                                    config_bpx['mean3g_mask'], reg3g_dir, \
                                    config_bpx['mean3g_config'])
        
        print('\t{}'.format(cmd))
        err = run_fsl(cmd)
        if err:
            print('\tError warping to Mean3G (fnirt) [{0}]: {1}'.format(subject,err))
            return False
        
        # Compute inverse warp
        cmd = '{0}invwarp --warp={1}/FA_warp2Mean3G.nii.gz ' \
                         '--out={1}/Mean3G_warp2FA.nii.gz ' \
                         '--ref={2}/dti_FA.nii.gz' \
                            .format(fsl_bin, reg3g_dir, output_dir)
                    
        print('\t{}'.format(cmd))
        err = run_fsl(cmd)
        if err:
            print('\tError warping to Mean3G (invwarp) [{0}]: {1}'.format(subject,err))
            return False
        else:
            print('\tDone computing transforms [{}].'.format(subject))
                
        
    
    return True

In [90]:
import csv

subjects = [];
os.chdir(current_dir)
with open(config_gen['subjects_file']) as subj_file:
    reader = csv.reader(subj_file)
    for row in reader:
        subjects.append(row[0])
                        
print('Processing {} subjects'.format(len(subjects)))

for subject in subjects:
    print('Processing subject {}'.format(subject))
    if process_subject(subject, config):
        print('Finished subject {}'.format(subject))
    else:
        print('Subject {} failed'.format(subject))

Processing 1 subjects
Processing subject A00037647
	BedpostX results already exist. Skipping those steps.
	Reg3D output exists for A00037647. Skipping.
Finished subject A00037647


In [24]:
import csv

def probtrackx_subject(subject, config):
    # Generate probabilistic streamlines between all pairs of ROIs
    # specified 
    
    # Get configs
    config_gen = config['general']
    config_bpx = config['bedpostx']
    config_ptx = config['probtrackx']
    
    fsl_bin = config_gen['fsl_bin']
    
    subj = '{}{}'.format(config_gen['prefix'], subject)
    
    raw_dir = '{0}/{1}'.format(config_gen['root_dir'], config_gen['raw_dir'])
    deriv_dir = '{0}/{1}'.format(config_gen['root_dir'], config_gen['deriv_dir'])
    
    # Subject-specific paths
    subj_dir = '{0}/{1}/{2}/dwi' \
                        .format(deriv_dir, subj, config_gen['session']) 
    bedpostx_dir = '{0}/bedpostX'.format(subj_dir)
    probtrackx_dir = '{0}/probtrackX/{1}'.format(subj_dir, config_ptx['network_name'])
    rois_dir = config_ptx['roi_dir']
    
    if not os.path.isdir(probtrackx_dir):
        os.makedirs(probtrackx_dir)
    
    invxfm_img = '{0}/reg3g/FA_warp2Mean3G.nii.gz'.format(subj_dir)
    xfm_img = '{0}/reg3g/Mean3G_warp2FA.nii.gz'.format(subj_dir)
    
    # Check whether BedpostX output exists, otherwise fail
    if not os.path.isdir(bedpostx_dir):
        print('No BedpostX output exists. Skipping subject. [{0}]'.format(subject)) 
        return False
    
    if not os.path.exists(xfm_img) or not os.path.exists(invxfm_img):
        print('No warp images (reg3G) exist. Skipping subject. [{0}]'.format(subject)) 
        return False
    
    # Read ROI list
    rois = []
    target_rois = {}
    
    # If this is a JSON file, read as networks and targets
    if config_ptx['roi_list'].endswith('.json'):
        with open(config_ptx['roi_list'], 'r') as myfile:
            json_string=myfile.read()

        netconfig = json.loads(json_string)
        print('Netconfig:')
        print(netconfig)
        networks = netconfig['networks']
        targets = netconfig['targets']

        for net in networks:
            others = []
            for net2 in targets[net]:
                others = others + networks[net2]
            for roi in networks[net]:
                print(roi)
                rois.append(roi)
                target_rois[roi] = others
    else:
        with open(config_ptx['roi_list'],'r') as roi_file:
            reader = csv.reader(roi_file)
            for row in reader:
                rois.append(row[0])
        
        # All ROIs are targets
        for roi in rois:
            targets = []
            for roi2 in rois:
                if roi is not roi2:
                    targets.append(roi2)
            target_rois[roi] = targets
    
    # For each ROI seed
    for roi in rois:
    
        roi_file = '{0}/{1}.nii'.format(rois_dir, roi)
        roi_list = '{0}/others_{1}.txt'.format(probtrackx_dir, roi)
    
        cmd_add = ''
    
        # Build list of other ROIs
        with open(roi_list,'w') as listout:
            for roi2 in target_rois[roi]:
                # Add to ROI list
                listout.write('{0}/{1}.nii\n'.format(rois_dir, roi2))
                # Add to "others" stop image
                if not cmd_add:
                    cmd_add = '{0}fslmaths {1}/{2}.nii'.format(fsl_bin, rois_dir, roi2)
                else:
                    cmd_add = '{0} -add {1}/{2}.nii'.format(cmd_add, rois_dir, roi2)
        
        stop_img = '{0}/{1}_others.nii.gz'.format(probtrackx_dir, roi)
        cmd_add = '{0} {1}'.format(cmd_add, stop_img)
            
        print(cmd_add)
        err = run_fsl(cmd_add)
        if err:
            print('\tError creating stop mask [{0}]: {1}'.format(subject,err))
            return False
    
#      common = [fullfile(pwd,newtrax) ' --distthresh=5.0 --sampvox=2  --forcedir --opd  --opathdir '  ...
#                 '-x ' Vi(r1).fname '  -l --onewaycondition -c 0.2 -S 2000 --steplength=0.5 -P 50000 --fibthresh=0.01 ' ...
#                 ' --xfm='    fullfile(multistate_dir,'DATA',NetFlags.SubDir{i},NetFlags.sub{i},'DWI','reg3G',fw) ...
#                 ' --invxfm=' fullfile(multistate_dir,'DATA',NetFlags.SubDir{i},NetFlags.sub{i},'DWI','reg3G',bk) ...
#                 ' -s ' fullfile(multistate_dir,'DATA',NetFlags.SubDir{i},NetFlags.sub{i},'DWI',[NetFlags.sub{i} '.bedpostX'],'merged') ...
#                 ' -m ' fullfile(multistate_dir,'DATA',NetFlags.SubDir{i},NetFlags.sub{i},'DWI',[NetFlags.sub{i} '.bedpostX'],'nodif_brain_mask')];
                
#         commonnet = [' --stop=' strrep(Vi(r1).fname,'.nii','_others.nii') ...
#                     ' --waypoints=' strrep(Vi(r1).fname,'.nii','_others.nii')  ' --waycond=OR ' ...
#                     ' --omatrix2 --target2=' strrep(Vi(r1).fname,'.nii','_others.nii') ...
#                     ' --os2t --targetmasks=' strrep(Vi(r1).fname,'.nii','_others.txt')  ' --otargetpaths' ];
    
#     probtrackx2 -x /Volumes/AndrewElements/data/rois/what-where/IPL_L_gwfa.nii -l 
#     --onewaycondition -c 0.2 -S 2000 --steplength=0.5 -P 5000 --fibthresh=0.01 
#     --distthresh=0.0 --sampvox=0.0 
#     --xfm=/Volumes/AndrewElements/data/nki/derivatives/sub-A00030989/ses-DS2/dwi/reg3G/Mean3G_warp2FA.nii.gz 
#     --invxfm=/Volumes/AndrewElements/data/nki/derivatives/sub-A00030989/ses-DS2/dwi/reg3G/FA_warp2Mean3G.nii.gz 
#     --forcedir --opd -s /Volumes/AndrewElements/data/nki/derivatives/sub-A00030989/ses-DS2/dwi/bedpostX/merged 
#     -m /Volumes/AndrewElements/data/nki/derivatives/sub-A00030989/ses-DS2/dwi/bedpostX/nodif_brain_mask 
#     --dir=/Volumes/AndrewElements/data/nki/derivatives/sub-A00030989/ses-DS2/dwi/ptx_test
# --opathdir ' 
    
    
        cmd_pre = '{0}probtrackx2 -V 0 --distthresh={1} --sampvox={2} --forcedir --opd ' \
                  '-x {3} -l --onewaycondition -c {4} -S {5} --steplength={6} -P {7} --fibthresh={8} ' \
                  '--xfm={9} --invxfm={10} -s {11}/merged -m {11}/nodif_brain_mask' \
                    .format(fsl_bin, config_ptx['distthresh'], config_ptx['sampvox'], roi_file, \
                           config_ptx['cthr'], config_ptx['nsteps'], config_ptx['steplength'], \
                           config_ptx['nsamples'], config_ptx['fibthresh'], xfm_img, invxfm_img, \
                           bedpostx_dir)

        cmd_net = ' --stop={0} -V 0 --waypoints={0} --waycond=OR --omatrix2 ' \
                  ' --target2={0} --os2t --targetmasks={1} --otargetpaths' \
                    .format(stop_img, roi_list)
                
        cmd = '{0} {1} --pd --dir={2}/{3}' \
                    .format(cmd_pre, cmd_net, probtrackx_dir, roi)
        print(cmd)
        err = run_fsl(cmd)
        if err:
            print('\tError running ProbtrackX [{0}]: {1}'.format(subject,err))
            return False
        
        cmd = '{0} {1} --pd --dir={2}/{3} -o FreeTracking' \
                    .format(cmd_pre, cmd_net, probtrackx_dir, roi)
        print(cmd)
        err = run_fsl(cmd)
        if err:
            print('\tError running ProbtrackX free-tracking [{0}]: {1}'.format(subject,err))
            return False
    
        print('\tDone tracking for ROI {0} [{1}]'.format(roi, subject))
        
    # Clean up
    cmd = 'rm {0}/others*; rm {0}/*others.nii.gz'.format(probtrackx_dir)
    err = run_fsl(cmd)
    if err:
        print('\tError cleaning up ProbtrackX folder [{0}]: {1}'.format(subject,err))
        return False
    
    
    return True

In [25]:
import csv

subjects = [];
os.chdir(current_dir)
with open(config_gen['subjects_file']) as subj_file:
    reader = csv.reader(subj_file)
    for row in reader:
        subjects.append(row[0])
            
# subjects=['A00030989']
            
print('Processing {} subjects'.format(len(subjects)))

for subject in subjects:
    print('Processing subject {}'.format(subject))
    if probtrackx_subject(subject, config):
        print('Finished subject {}'.format(subject))
    else:
        print('Subject {} failed'.format(subject))

Processing 1 subjects
Processing subject A00037647
/usr/local/fsl/bin/fslmaths /Volumes/AndrewElements/data/rois/cing_rs/B8.nii -add /Volumes/AndrewElements/data/rois/cing_rs/B8a.nii -add /Volumes/AndrewElements/data/rois/cing_rs/B8b.nii -add /Volumes/AndrewElements/data/rois/cing_rs/B8c.nii -add /Volumes/AndrewElements/data/rois/cing_rs/B9.nii -add /Volumes/AndrewElements/data/rois/cing_rs/BG_GM.nii -add /Volumes/AndrewElements/data/rois/cing_rs/Cingulate_GM.nii -add /Volumes/AndrewElements/data/rois/cing_rs/Cingulate_revised.nii /Volumes/AndrewElements/data/nki/derivatives/sub-A00037647/ses-DS2/dwi/probtrackX/cing_rs/B46_others.nii.gz
/usr/local/fsl/bin/probtrackx2 -V 0 --distthresh=5.0 --sampvox=2 --forcedir --opd -x /Volumes/AndrewElements/data/rois/cing_rs/B46.nii -l --onewaycondition -c 0.2 -S 2000 --steplength=0.5 -P 5000 --fibthresh=0.01 --xfm=/Volumes/AndrewElements/data/nki/derivatives/sub-A00037647/ses-DS2/dwi/reg3g/Mean3G_warp2FA.nii.gz --invxfm=/Volumes/AndrewElements/data

	Done tracking for ROI B8a [A00037647]
/usr/local/fsl/bin/fslmaths /Volumes/AndrewElements/data/rois/cing_rs/B46.nii -add /Volumes/AndrewElements/data/rois/cing_rs/B8.nii -add /Volumes/AndrewElements/data/rois/cing_rs/B8a.nii -add /Volumes/AndrewElements/data/rois/cing_rs/B8c.nii -add /Volumes/AndrewElements/data/rois/cing_rs/B9.nii -add /Volumes/AndrewElements/data/rois/cing_rs/BG_GM.nii -add /Volumes/AndrewElements/data/rois/cing_rs/Cingulate_GM.nii -add /Volumes/AndrewElements/data/rois/cing_rs/Cingulate_revised.nii /Volumes/AndrewElements/data/nki/derivatives/sub-A00037647/ses-DS2/dwi/probtrackX/cing_rs/B8b_others.nii.gz
/usr/local/fsl/bin/probtrackx2 -V 0 --distthresh=5.0 --sampvox=2 --forcedir --opd -x /Volumes/AndrewElements/data/rois/cing_rs/B8b.nii -l --onewaycondition -c 0.2 -S 2000 --steplength=0.5 -P 5000 --fibthresh=0.01 --xfm=/Volumes/AndrewElements/data/nki/derivatives/sub-A00037647/ses-DS2/dwi/reg3g/Mean3G_warp2FA.nii.gz --invxfm=/Volumes/AndrewElements/data/nki/derivat

	Done tracking for ROI B9 [A00037647]
/usr/local/fsl/bin/fslmaths /Volumes/AndrewElements/data/rois/cing_rs/B46.nii -add /Volumes/AndrewElements/data/rois/cing_rs/B8.nii -add /Volumes/AndrewElements/data/rois/cing_rs/B8a.nii -add /Volumes/AndrewElements/data/rois/cing_rs/B8b.nii -add /Volumes/AndrewElements/data/rois/cing_rs/B8c.nii -add /Volumes/AndrewElements/data/rois/cing_rs/B9.nii -add /Volumes/AndrewElements/data/rois/cing_rs/Cingulate_GM.nii -add /Volumes/AndrewElements/data/rois/cing_rs/Cingulate_revised.nii /Volumes/AndrewElements/data/nki/derivatives/sub-A00037647/ses-DS2/dwi/probtrackX/cing_rs/BG_GM_others.nii.gz
/usr/local/fsl/bin/probtrackx2 -V 0 --distthresh=5.0 --sampvox=2 --forcedir --opd -x /Volumes/AndrewElements/data/rois/cing_rs/BG_GM.nii -l --onewaycondition -c 0.2 -S 2000 --steplength=0.5 -P 5000 --fibthresh=0.01 --xfm=/Volumes/AndrewElements/data/nki/derivatives/sub-A00037647/ses-DS2/dwi/reg3g/Mean3G_warp2FA.nii.gz --invxfm=/Volumes/AndrewElements/data/nki/deriva

KeyboardInterrupt: 